In [2]:
import os
import pandas as pd

subset_folder = "/Volumes/T7/OMSCS/CLEF2025/EXIST2025/exist-2025/notebooks/gemini/subsets_trial5"
csv_files = [f for f in os.listdir(subset_folder) if f.endswith('.csv')]

dfs = []
for file in csv_files:
    file_path = os.path.join(subset_folder, file)
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        print(f"⚠️ UTF-8 failed for {file}, trying ISO-8859-1...")
        df = pd.read_csv(file_path, encoding='ISO-8859-1')
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)
df_all = df_all.loc[:, ~df_all.columns.str.startswith('Unnamed:')]
df_all = df_all[df_all['description'] != 'ERROR']
df_all['id_EXIST'] = df_all['id_EXIST'].astype('Int64')
df_all = df_all[df_all['id_EXIST'].notna()]

In [6]:
# Read the train, valid, and test dataframes
train_df = pd.read_csv('/Volumes/T7/OMSCS/CLEF2025/EXIST2025/exist-2025/notebooks/train_test_split/train_df.csv')
valid_df = pd.read_csv('/Volumes/T7/OMSCS/CLEF2025/EXIST2025/exist-2025/notebooks/train_test_split/valid_df.csv')
test_df = pd.read_csv('/Volumes/T7/OMSCS/CLEF2025/EXIST2025/exist-2025/notebooks/train_test_split/test_df.csv')

# Merge each dataframe separately with df_all
train_merged = pd.merge(
    train_df,
    df_all[['id_EXIST', 'description', 'analysis']].rename(
        columns={'description': 'description_fp', 'analysis': 'analysis_fp'}
    ),
    on='id_EXIST',
    how='left'
)

valid_merged = pd.merge(
    valid_df,
    df_all[['id_EXIST', 'description', 'analysis']].rename(
        columns={'description': 'description_fp', 'analysis': 'analysis_fp'}
    ),
    on='id_EXIST',
    how='left'
)

test_merged = pd.merge(
    test_df,
    df_all[['id_EXIST', 'description', 'analysis']].rename(
        columns={'description': 'description_fp', 'analysis': 'analysis_fp'}
    ),
    on='id_EXIST',
    how='left'
)

# Display the first few rows of each merged dataframe
print("Train merged:")
print(train_merged.head())
print("\nValid merged:")
print(valid_merged.head())
print("\nTest merged:")
print(test_merged.head())



Train merged:
             id_Tiktok  id_EXIST lang  \
0  6920327322679692545    220242   en   
1  6935046770778967302    220296   en   
2  7123342389338443009    220920   en   
3  7076153533996928302    220768   en   
4  7039365996053794053    220665   en   

                                                text                    video  \
0  these men stay  sparkles pressed sparkles  che...  6920327322679692545.mp4   
1   beaming_face_with_smiling_eyes  ‘school’ pent...  6935046770778967302.mp4   
2  don’t laugh at your girlfriends choice, you ar...  7123342389338443009.mp4   
3  i guess she’s not wrong  face_with_tears_of_jo...  7076153533996928302.mp4   
4  women, why do we do this?  woman_facepalming_m...  7039365996053794053.mp4   

                       path_video  \
0  videos/6920327322679692545.mp4   
1  videos/6935046770778967302.mp4   
2  videos/7123342389338443009.mp4   
3  videos/7076153533996928302.mp4   
4  videos/7039365996053794053.mp4   

                             

In [13]:
# Get IDs where description_fp is null in each dataframe
train_null_ids = train_merged[train_merged['description_fp'].isna()]['id_EXIST'].tolist()
valid_null_ids = valid_merged[valid_merged['description_fp'].isna()]['id_EXIST'].tolist()
test_null_ids = test_merged[test_merged['description_fp'].isna()]['id_EXIST'].tolist()

# Print the results
print("Train null IDs:", len(train_null_ids))
print(train_null_ids)
print("\nValid null IDs:", len(valid_null_ids))
print(valid_null_ids)
print("\nTest null IDs:", len(test_null_ids))
print(test_null_ids)


Train null IDs: 3
[220013, 220057, 220530]

Valid null IDs: 0
[]

Test null IDs: 161
[220417, 220757, 220986, 220304, 220795, 220447, 220267, 220896, 220338, 220343, 220654, 220574, 220453, 220126, 220606, 220333, 220718, 220612, 220041, 220811, 220977, 220320, 220324, 220062, 220911, 220511, 220439, 220742, 220556, 220904, 220496, 220972, 220418, 220074, 220142, 220317, 220871, 220729, 220782, 220659, 220992, 220066, 220826, 220369, 220605, 220929, 220309, 220360, 220372, 220245, 220361, 220477, 220735, 220308, 220091, 220935, 220526, 220300, 220113, 220996, 220262, 220223, 220563, 220329, 220588, 220733, 220681, 220813, 220553, 220849, 220820, 220532, 220058, 220616, 220226, 220503, 220076, 220546, 220046, 220270, 220217, 220724, 220960, 220547, 220287, 220552, 220239, 220738, 220907, 220778, 220713, 220647, 220319, 220821, 220905, 220832, 220938, 220221, 220499, 220063, 220454, 220700, 220442, 220145, 220597, 220625, 220290, 220321, 220497, 220463, 220116, 220450, 220776, 220326, 22

In [14]:
data_path = '/Users/moiz.ali/Downloads/EXIST 2025 Dataset V0.3/EXIST 2025 Videos Dataset/training/EXIST2025_training.json'

In [ ]:
df = pd.read_json(data_path).T

# Combine train and test null IDs
common_null_ids = train_null_ids + test_null_ids

# Filter the dataframe to only include rows where the index is in common_null_ids
df_filtered = df[df.index.isin(common_null_ids)]
df_filtered.shape

df = df_filtered

(164, 14)

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)
model_name = "gemini-2.5-pro-preview-05-06" # @param ["gemini-1.5-flash-latest","gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-flash-preview-04-17","gemini-2.5-pro-exp-03-25"] {"allow-input":true, isTemplate: true}
import time
import pandas as pd

# Assuming `client` and `model_name` are already set up
# df is your DataFrame with columns: text, path_video, target

def upload_video(video_file_name):
    video_file = client.files.upload(file=video_file_name)

    while video_file.state == "PROCESSING":
        print(f'Processing {video_file_name}...')
        time.sleep(10)
        video_file = client.files.get(name=video_file.name)

    if video_file.state == "FAILED":
        raise ValueError(f'Video processing failed for {video_file_name}')
    
    print(f'Video processing complete: {video_file.uri}')
    return video_file

# Prompt for sexism analysis
# sexism_prompt = (
#     "You are an expert in identifying the presence of sexism within the video. "
#     "Your task is to assess whether the content of the video criticizes through marginalization, "
#     "prejudice against women based on gender stereotypes, beliefs in male superiority, or misogynistic views. "
#     "Briefly describe and analyze the content of the video then label it as 'YES' if the video embodies or criticizes sexism, "
#     "'NO' if it does not. Please use the following format. "
#     "1. Description [Your description] 2. Label [YES/NO]"
# )

sexism_prompt = (
    "You are an expert in identifying the presence of sexism in video content. "
    "Your task is to assess whether the content of the video criticizes through marginalization, "
    "prejudice against women based on gender stereotypes, beliefs in male superiority, or misogynistic views."
    "Briefly describe and analyze the content of the video then label it as 'YES' if the video embodies or criticizes sexism."
    "Please respond strictly in the following JSON format:\n\n"
    '{\n'
    '  "description": "[Your one-sentence description of the video]",\n'
    '  "label": "YES" or "NO"\n'
    '  "analysis": [One or two sentences explaining why the video is or isnt sexist]"\n'
    '}\n\n'
    "Only return valid JSON. Do not include any explanations or extra text."
)

# sexism_prompt = (
#     "You are an expert in detecting sexism in video content. Take the role of a female"
#     "Your task is to assess whether the video either displays or critiques sexist content. "
#     "You will first be shown several example videos that have already been labeled as either sexist or not sexist, based on clear criteria."
#     "Sexism is considered present if the content falls into any of the following five categories: "
#     "(i) IDEOLOGICAL AND INEQUALITY — promoting or reinforcing beliefs about male superiority or social inequality between genders, "
#     "(ii) STEREOTYPING AND DOMINANCE — reinforcing traditional gender roles or power imbalance, "
#     "(iii) OBJECTIFICATION — reducing a person (typically a woman) to their body or physical appearance, "
#     "(iv) SEXUAL VIOLENCE — explicit or implied depictions of harassment, assault, or coercion, "
#     "(v) MISOGYNY AND NON-SEXUAL VIOLENCE — expressing hatred or aggression toward women in non-sexual ways. "
#     "You must be **extra critical and cautious** in your assessment. Ultimately, the judgement criteria used in the example videos is the one you should use to label the new videos."
#     "Please respond strictly in the following JSON format:\n\n"
#     '{\n'
#     '  "description": "[Your one-sentence description of the video]",\n'
#     '  "label": "YES" or "NO",\n'
#     '  "analysis": "[One or two sentences explaining why the video is or isnt sexist, with reference to one or more of the five categories]"\n'
#     '}\n\n'
#     "Only return valid JSON. Do not include any explanations, category names, or extra text outside the JSON."
# # )

# sexism_prompt = (
#     """
#     You are an expert in identifying sexism in video content. Your primary task is to distinguish between videos that *are themselves sexist* (i.e., they actively promote, endorse, or normalize sexism through their own message, framing, or overall presentation) and videos that merely *depict, discuss, or incidentally include* sexist content (e.g., for critique, satire, factual reporting, artistic expression, or as part of a narrative where the video *itself* does not endorse the sexism). Focus on what the *video creator* is trying to convey as their own viewpoint or message, not just what is shown or said within the video.

#     **The core question is: Does the video *itself* actively promote, endorse, or normalize sexism, intending to persuade the viewer of sexist ideas?**

#     Sexism is defined as the *video itself* communicating one or more of the following, in a way that the video *actively promotes or clearly endorses as acceptable, normal, or true*, thereby contributing to inequality or harm:

#     *   **Marginalization:** The video *itself* presents a gender (typically women) as inherently less important, capable, or deserving, or unfairly limits their opportunities or voice *due to their gender*, in a manner the video appears to support or normalize.
#     *   **Harmful Stereotypes:** The video *itself* actively promotes demeaning, limiting, or broadly negative generalizations about a gender *as if they are inherent truths*. For this to be sexist, the video's discernible primary intent in promoting such stereotypes must be to reinforce inequality, belittle the gender, or restrict their opportunities. Observations or humor based on gender are *not* sexist under this rule *unless the video's overall framing and primary intent clearly weaponize these stereotypes to achieve such harmful outcomes as its central message.* (e.g., simply listing stereotypical "wants" of women is not sexist unless the video uses this to argue for their inferiority or to justify discriminatory treatment).
#     *   **Male Superiority:** The video *itself* asserts or implies men's inherent dominance, greater value, or rightful control over women, framing this as natural, desirable, or justified.
#     *   **Misogyny:** The video *itself*, through its own primary message or creative choices, expresses or encourages hatred, contempt, or ingrained prejudice against women. The mere inclusion of misogynistic language (e.g., in song lyrics, character dialogue) does *not* automatically make the video misogynistic *unless the video's own framing and primary intent clearly center on endorsing, celebrating, or amplifying that misogynistic sentiment as its own message.*

#     **Crucial Decision Point (Labeling):**

#     *   **Label "YES"**: If the video's *own discernible primary message, tone, narrative voice, or overall presentation* clearly and actively promotes, endorses, normalizes, or celebrates any of the sexist elements defined above. The video itself is the source of sexist advocacy or validation.
#     *   **Label "NO"**:
#         *   If the video depicts sexist acts, language, or ideas *primarily to critique, condemn, satirize, or factually report on them*, where the video's own stance is clearly against the depicted sexism or unendorsing of it.
#         *   If sexist ideas/actions are expressed by characters or are part of a narrative, but the *video itself does not demonstrably endorse or promote these as valid, acceptable, or desirable as part of its own primary message*. The video might be exploring complex themes or showing flawed characters without its own voice condoning the sexism.
#         *   If the video depicts common or lighthearted gender stereotypes without the video's *own discernible primary intent* being to use these stereotypes to demean, restrict, or advocate for unequal treatment of a gender. The video isn't *weaponizing* the stereotype to push a harmful sexist agenda.
#         *   If the video incorporates material containing sexist language or ideas (e.g., song lyrics, dialogue from a film) but the *video's own primary focus, message, and creative intent* are not to endorse or amplify the sexism within that material. The presence of such material is incidental to, or serves a different purpose within, the video's overall non-sexist message (e.g., used for its beat, a non-sexist thematic element, or artistic quotation).
#         *   If sexist elements are merely incidental background elements not central to any message actively endorsed by the video, and are not the focus of the video's *own* active promotion or endorsement.

#     Respond strictly in the following JSON format:
#     {
#     "description": "[One sentence describing the video's relevant content AND, crucially, the video's *own apparent stance or framing* of that content, focusing on whether the video *itself* promotes, endorses, or normalizes sexism.]",
#     "label": "YES" or "NO"
#     "analysis": "[One sentence describing the reason why it was labelled as YES or NO, referencing the specific definitions if applicable]"
#     "probability": Give a number between 0 and 1 based on how confident you are on the label where >= 0.5 and < 1 implies a label of "YES" and between 0 and 0.5 implies a label of NO
#     }

#     Only return valid JSON. Do not include any explanations or extra text.
#     """
# )

# sexism_prompt = (
#     """
#     You are an expert in identifying sexism in video content.
#     Your task is to determine if the video *itself is sexist*.

#     **Definition of Sexism for this task:**
#     Sexism involves actions or expressions that promote, endorse, or normalize:
#     1.  Marginalization of individuals based on sex/gender.
#     2.  Harmful gender stereotypes.
#     3.  The idea of male superiority.
#     4.  Misogyny (e.g., hatred, contempt, denigration of women, or the video's own uncritical use of misogynistic slurs or depiction of sexual objectification as positive/normal).

#     **Labeling Guidance (Focus on the video's *own apparent stance or framing*):**

#     -   **Label "YES" if:** The video *itself* actively promotes, endorses, or normalizes sexism (as defined above).
#     -   **Label "NO" if:**
#         -   The video *critiques, condemns, satirizes, or exposes* sexism, even if sexist elements are depicted or discussed. The video's aim is to challenge sexism.
#         -   Sexist elements are present (e.g., in background music, quoted material, depicted character behavior) but are *incidental* to the video's primary message. The video does not amplify, endorse, or leverage these elements to make its own sexist point, and the video as a whole does not promote sexism. (e.g., a dance tutorial using a song with sexist lyrics, where the video focuses purely on dance and doesn't endorse the lyrics).
#         -   The video does not contain or engage with sexism in any significant way.

#     Consider all video elements: visuals, audio, spoken words, and on-screen text.
    
#     {
#     "description": "[One sentence describing the video's relevant content AND, crucially, the video's *own apparent stance or framing* of that content – does it promote/endorse sexism, critique it, or are sexist elements incidental and unendorsed?]",
#     "label": "YES" or "NO",
#     "analysis": "[One sentence explaining *why the video itself is or is not sexist*, referencing its stance/framing and how it handles any sexist content. E.g., 'The video promotes harmful stereotypes about women.' OR 'The video critiques misogyny by exposing its absurdity.' OR 'Sexist song lyrics are incidental and unendorsed by the video's primary focus on [topic].']",
#     "probability": "Give a number between 0 and 1. >= 0.5 implies 'YES', < 0.5 implies 'NO'."
#     }
#     Only return valid JSON. Do not include any explanations or extra text.
#     """
# )

# Store outputs
descriptions = []
labels = []
text = []
analysis_lst = []

df.head()
# Read trial_fp.csv from the correct path
# df_trial_fp = pd.read_csv("/Volumes/T7/OMSCS/CLEF2025/EXIST2025/exist-2025/notebooks/gemini/eval_results/trial1_fp.csv")
# exist_ids = df_trial_fp['id_EXIST'].tolist()

# Filter main dataframe based on EXIST IDs from trial_fp
# df_all = df_all[df_all['id_EXIST'].isin(exist_ids)]

# df = df_all
df.shape
print(sexism_prompt)
import os
import json
import pandas as pd
from tqdm import tqdm
import numpy as np

# Parameters
num_subsets = 20  # change as needed
subset_folder = "subsets_test_fn"
os.makedirs(subset_folder, exist_ok=True)

# Split the dataset
df_split = np.array_split(df, num_subsets)

for i, df_subset in enumerate(df_split):
    print(f"\n🚀 Processing subset {i+1}/{num_subsets} with {len(df_subset)} rows...")

    file_path = os.path.join(subset_folder, f"subset_{i+1:02d}.csv")
    print(file_path)

    if os.path.exists(file_path):

        pass
    
    else:

        descriptions = []
        labels = []
        analysis_lst = []

        for idx, row in tqdm(df_subset.iterrows(), total=len(df_subset)):
            try:
                path = f"/Users/moiz.ali/Downloads/EXIST 2025 Dataset V0.3/EXIST 2025 Videos Dataset/test/{row['path_video']}"
                uploaded_video = upload_video(path)

                response = client.models.generate_content(
                    model=model_name,
                    contents=[uploaded_video, sexism_prompt]
                )

                result = json.loads(response.text.strip().strip("```json").strip("```"))
                description = result.get("description", "ERROR")
                label = result.get("label", "ERROR")
                analysis = result.get("analysis", "ERROR")
            
            except Exception as e:
                print(f"❌ Error at index {idx} (id {row['id_EXIST']}): {e}")
                description = "ERROR"
                label = "ERROR"
                analysis = "ERROR"

            descriptions.append(description)
            labels.append(label)
            analysis_lst.append(analysis)

        # Save results for this subset
        df_subset['description'] = descriptions
        df_subset['label'] = labels
        df_subset['analysis'] = analysis_lst

        output_path = os.path.join(subset_folder, f"subset_{i+1:02d}.csv")
        df_subset.to_csv(output_path, index=False)
        print(f"💾 Saved results to: {output_path}")


print("\n✅ All subsets processed and saved.")


In [12]:
print("Train DataFrame shape:", train_df.shape)
print("Train Merged DataFrame shape:", train_merged.shape)
print("Number of non-null descriptions in train_merged:", train_merged['description_fp'].notna().sum())
print("\nValid DataFrame shape:", valid_df.shape)
print("Valid Merged DataFrame shape:", valid_merged.shape)
print("Number of non-null descriptions in valid_merged:", valid_merged['description_fp'].notna().sum())
print("\nTest DataFrame shape:", test_df.shape)
print("Test Merged DataFrame shape:", test_merged.shape)
print("Number of non-null descriptions in test_merged:", test_merged['description_fp'].notna().sum())



Train DataFrame shape: (583, 15)
Train Merged DataFrame shape: (583, 17)
Number of non-null descriptions in train_merged: 580

Valid DataFrame shape: (195, 15)
Valid Merged DataFrame shape: (195, 17)
Number of non-null descriptions in valid_merged: 195

Test DataFrame shape: (195, 15)
Test Merged DataFrame shape: (195, 17)
Number of non-null descriptions in test_merged: 34
